##

sub-21
File "/home/mrenke/git/numrisk/numrisk/fmri_analysis/encoding_model/fit_nprf_cv_stim2.py", line 136, in <module>
main(args.subject, bids_folder=args.bids_folder, smoothed=args.smoothed, retroicor=args.retroicor,
File "/home/mrenke/git/numrisk/numrisk/fmri_analysis/encoding_model/fit_nprf_cv_stim2.py", line 70, in main
`data = pd.DataFrame(masker.fit_transform(data), index=paradigm.index)`
File "/data/mrenke/conda/envs/numrefields/lib/python3.9/site-packages/pandas/core/frame.py", line 721, in __init__
mgr = ndarray_to_mgr(
File "/data/mrenke/conda/envs/numrefields/lib/python3.9/site-packages/pandas/core/internals/construction.py", line 349, in ndarray_to_mgr
_check_values_indices_shape_match(values, index, columns)
File "/data/mrenke/conda/envs/numrefields/lib/python3.9/site-packages/pandas/core/internals/construction.py", line 420, in _check_values_indices_shape_match
raise ValueError(f"Shape of passed values is {passed}, indices imply {implied}")
ValueError: Shape of passed values is (105, 74624), indices imply (180, 74624)

sub-3
....    _check_values_indices_shape_match(values, index, columns)
  File "/data/mrenke/conda/envs/numrefields/lib/python3.9/site-packages/pandas/core/internals/construction.py", line 420, in _check_values_indices_shape_match
    raise ValueError(f"Shape of passed values is {passed}, indices imply {implied}")
ValueError: Shape of passed values is (104, 84111), indices imply (180, 84111)


###
for sub 3 this error comes already for fit_glm_stim1 ! (not stim2 though...)
different, thoug:
  File "/home/mrenke/git/numrisk/numrisk/fmri_analysis/glm_denoise/fit_glm_denoise.py", line 96, in main
    results_glmsingle = glmsingle_obj.fit(
  File "/home/mrenke/git/GLMsingle/glmsingle/glmsingle.py", line 608, in fit
    results0 = glm_estimatemodel(
  File "/home/mrenke/git/GLMsingle/glmsingle/ols/glm_estimatemodel.py", line 425, in glm_estimatemodel
    data, design, xyz = check_inputs(data, design)
  File "/home/mrenke/git/GLMsingle/glmsingle/check_inputs.py", line 27, in check_inputs
    np.testing.assert_array_equal(
  File "/data/mrenke/conda/envs/numrefields/lib/python3.9/site-packages/numpy/testing/_private/utils.py", line 934, in assert_array_equal
    assert_array_compare(operator.__eq__, x, y, err_msg=err_msg,
  File "/data/mrenke/conda/envs/numrefields/lib/python3.9/site-packages/numpy/testing/_private/utils.py", line 763, in assert_array_compare
    raise AssertionError(msg)
AssertionError:
Arrays are not equal
<design> must consist of 0s and 1s
(shapes (3,), (2,) mismatch)
 x: array([0., 1., 2.])
 y: array([0, 1])


In [1]:
import argparse
import pandas as pd
from braincoder.models import GaussianPRF
from braincoder.optimize import ParameterFitter
from braincoder.utils import get_rsq
from numrisk.utils import get_target_dir
from nilearn.input_data import NiftiMasker

import os.path as op
import numpy as np

2024-10-22 14:04:17.730150: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
/data/mrenke/conda/envs/numrefields/lib/python3.9/site-packages/nilearn/input_data/__init__.py:23: FutureWarning: The import path 'nilearn.input_data' is deprecated in version 0.9. Importing from 'nilearn.input_data' will be possible at least until release 0.13.0. Please import from 'nilearn.maskers' instead.
  warnings.warn(message, FutureWarning)


In [26]:
bids_folder = '/shares/zne.uzh/mrenke/ds-dnumrisk'

subject = 3


subject = f'{int(subject):02d}'
session = 1
runs = range(1, 7)
n_stim = 2
key = f'glm_stim{n_stim}'
target_dir = f'encoding_model_stim{n_stim}.cv'
key += '.denoise'
target_dir += '.denoise'


In [27]:
target_dir = get_target_dir(subject, session, bids_folder, target_dir)

paradigm = [pd.read_csv(op.join(bids_folder, f'sub-{subject}', f'ses-{session}',
                                'func', f'sub-{subject}_ses-{session}_task-magjudge_run-{run}_events.tsv'), sep='\t')
            for run in runs]
paradigm = pd.concat(paradigm, keys=runs, names=['run'])
paradigm = paradigm[paradigm.trial_type == f'stimulus {n_stim}'].set_index('trial_nr', append=True)

paradigm[f'log(n{n_stim})'] = np.log(paradigm[f'n{n_stim}'])
paradigm = paradigm[f'log(n{n_stim})'].dropna()

In [28]:
paradigm

run      trial_nr
1    1   1           1.386294
     4   2           1.609438
     7   3           1.386294
     10  4           2.397895
     13  5           2.397895
                       ...   
6    76  176         3.496508
     79  177         2.197225
     82  178         2.639057
     85  179         1.098612
     88  180         2.397895
Name: log(n2), Length: 180, dtype: float64

In [29]:
model = GaussianPRF()
# SET UP GRID
mus = np.log(np.linspace(5, 80, 60, dtype=np.float32))
sds = np.log(np.linspace(2, 30, 60, dtype=np.float32))
amplitudes = np.array([1.], dtype=np.float32)
baselines = np.array([0], dtype=np.float32)

mask = op.join(bids_folder, 'derivatives',
                f'fmriprep/sub-{subject}/ses-{session}/func/sub-{subject}_ses-{session}_task-magjudge_run-1_space-T1w_desc-brain_mask.nii.gz')
if not op.exists(mask): # for some subs brain mask creation for some runs did not work apparently, so lets just take another run (they should be pretty similar?)
    mask = op.join(bids_folder, 'derivatives',
            f'fmriprep/sub-{subject}/ses-{session}/func/sub-{subject}_ses-{session}_task-magjudge_run-2_space-T1w_desc-brain_mask.nii.gz')

masker = NiftiMasker(mask_img=mask)



In [30]:
data = op.join(bids_folder, 'derivatives', key,
                f'sub-{subject}', f'ses-{session}', 'func', f'sub-{subject}_ses-{session}_task-magjudge_space-T1w_desc-stims{n_stim}_pe.nii.gz')

from nilearn import image

ims = image.load_img(data)

In [31]:
ims.get_fdata().shape

(58, 73, 50, 104)

In [32]:
tmp = masker.fit_transform(data)

In [21]:
tmp

array([[ -5.586601  ,  -0.25460613,  13.430651  , ...,   3.276642  ,
         -0.39079255,  -9.162464  ],
       [  0.17910606,   6.005245  ,  19.335033  , ...,   3.3928962 ,
         -1.3308599 ,   4.1230335 ],
       [  5.7466693 ,   1.1067244 , -10.965698  , ...,  -1.0369884 ,
         -2.8626223 ,   7.8606257 ],
       ...,
       [ -1.7437749 ,   3.8153033 ,   2.5822282 , ...,  -6.538029  ,
         -0.23672003, -41.86214   ],
       [ -7.4950514 ,  -5.620179  ,   1.7101481 , ...,  28.328648  ,
         -6.5455327 ,  88.55697   ],
       [ -3.7700734 ,   7.8073134 , -23.333174  , ...,  -6.3825936 ,
         18.216936  ,  25.894047  ]], dtype=float32)

In [ ]:
data = pd.DataFrame(tmp, index=paradigm.index)
